In [1]:
import time
import numpy as np
import pandas as pd
import math
import scipy.stats as ss
from scipy.special import chdtri
start = time.time()

In [2]:
class Sardines:
    
    def __init__(self):
        self.k = int(2)
        self.alpha = float(0.1)
        self.data = pd.read_csv("sardines.csv", sep = ";")
        self.chisq, self.avg, self.std, self.chi_alpha= 0, 0, 0, 0
        self.u, self.F, self.p, self.np, self.sqrs, self.sqrs_div = [], [], [], [], [], []
        self.dof = self.data.shape[0] - self.k - 1
        self.main()
        
    def average(self):
        sum_avg = 0
        for i in range(self.data.shape[0]):
            x = (1 / 2) * (self.data.iloc[i, 0] + self.data.iloc[i, 1]) * self.data.iloc[i, 2]
            sum_avg += x
        self.avg = sum_avg / sum(self.data.iloc[:,2])
        print("Average:", self.avg)
        
    def sd(self):
        self.average()
        sum_avg_std = 0
        for i in range(self.data.shape[0]):
            x = math.pow(((1 / 2) * (self.data.iloc[i, 0] + self.data.iloc[i, 1]) - self.avg), 2) * self.data.iloc[i, 2]
            sum_avg_std += x
        self.std = math.sqrt(sum_avg_std / sum(self.data.iloc[:, 2]))
        print("Standard deviation:", self.std)
        
    def u_values(self):
        self.sd()
        for i in range(self.data.shape[0] - 1):
            u_val = (self.data.iloc[i, 1] - self.avg) / self.std
            self.u.append(u_val)
        self.u.append("-")
    
    def F_values(self):
        self.u_values()
        for i in range(self.data.shape[0] - 1):
            F_val = ss.norm.cdf(self.u[i], loc=0, scale=1)
            self.F.append(F_val)
        self.F.append("-")
        
    def p_values(self):
        self.F_values()
        sum = 0
        self.p.append(self.F[0])
        for i in range(1, self.data.shape[0] - 1):
            value = self.F[i] - self.F[i - 1]
            self.p.append(value)
        for i in range(self.data.shape[0] - 1):
            sum += self.p[i]
        last = 1 - sum
        self.p.append(last)
    
    def sqr_values(self):
        self.p_values()
        n = 0
        for i in range(self.data.shape[0]):
            n += self.data.iloc[i, self.data.shape[1]-1]
        for i in range(self.data.shape[0]):
            self.np.append(n * self.p[i])
        for i in range(self.data.shape[0]):
            self.sqrs.append((self.data.iloc[i, 2] - self.np[i]) ** 2)
        for i in range(self.data.shape[0]):
            self.sqrs_div.append((self.sqrs[i] / self.np[i]))
            
    def chi_sq(self):
        self.sqr_values()
        for i in range(self.data.shape[0]):
            self.chisq += self.sqrs_div[i]
        self.chi_alpha = chdtri(1 - self.alpha, self.dof)
        print(self.chi_alpha)#6.251389
        
    def main(self):
        self.chi_sq()

sard = Sardines()
print("Time of work:", round((time.time() - start), 2) ,'s')

Average: 16.2
Standard deviation: 2.4657656011875906
nan
Time of work: 0.03 s
